# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    # Write your code here
    
    total = np.sum(P['Amount Invested'])
    
    erp = 0
    
    for i in range(len(P)) :
        er = rf + mkt_rp*P['Beta'].iloc[i]
        erp += er*P['Amount Invested'].iloc[i]/total
    
    return erp

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [149]:
def volatility(P, cor):
    # Write your code here
    
    total = np.sum(P['Amount Invested'])
    pvar = 0
    
    for i in range(len(P)) :
        
        pvar += (P['Volatility'].iloc[i]*P['Amount Invested'].iloc[i]/total)**2
        
        for v in range(i+1, len(P)) :
            pvar += 2*cor*P['Volatility'].iloc[i]*(P['Amount Invested'].iloc[i]/total)*P['Volatility'].iloc[v]*(P['Amount Invested'].iloc[v]/total)
        
#     for i in range(1, len(P)) :
#         prod *= cor*(P['Volatility'].iloc[i])*(P['Volatility'].iloc[i-1])
    
    return np.sqrt(pvar)

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [22]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    # Write your code here
    
    x = vol / mkt_vol
    
    er = rf + mkt_rp*x
    
    return er


### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [257]:
def ar(data) :
    return np.mean(data['RET']*100)

In [261]:
def bt(data, fdf) :
    
    fdf['date'] = fdf['date'].astype(str)
    data['date1'] = data['year'].astype(str)+data['month'].astype(str).str.zfill(2)
    data['date1'] = data['date1'].astype(str)
    data = pd.merge(data,fdf,how='left',left_on='date1',right_on='date').drop(['date1'], axis=1)
    
    data['R-RF'] = data['RET']*100-data['RF']
    RRFstd = np.std(data['R-RF'])
    cor = np.corrcoef(data['R-RF'], data['Mkt-RF'])[0][1]
    MRFstd = np.std(data['Mkt-RF'])
    
    return RRFstd*cor/MRFstd

In [262]:
def CAPM_beta(return_path, factors_path):
    #Write your code here
    output = pd.DataFrame(data = {
        "CUSIP" : [] ,
        "EXP_RETURN" : [],
        "BETA" : []
    })
    
    rdf = pd.read_csv(return_path)
    fdf = pd.read_csv(factors_path)
    
    grdf = rdf.groupby('CUSIP')
    avg_R = grdf.apply(ar)
    beta = grdf.apply(bt, fdf)
    
    output['CUSIP'] = rdf['CUSIP'].drop_duplicates()
    output = output.set_index('CUSIP')
    output.loc[avg_R.index , 'EXP_RETURN'] = avg_R
    output.loc[beta.index , 'BETA'] = beta
    output = output.reset_index()
    
    return output

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [264]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    return np.polyfit(data['BETA'], data['EXP_RETURN'], 1)[1]